[last week's code](../w9/varCode.ipynb)

In [74]:
import pandas as pd
from tqdm import tqdm, trange

In [59]:
bw=pd.read_csv("../w9/bw0801.csv")
ri=pd.read_csv('D:/V2_RISK_INCIDENTS.csv')

In [77]:
def strYear2Num(s):
    if s=="" or s=="nan":
        return s
    return s.split("/")[-1][:4]+s.split("/")[1].rjust(2,"0")+s.split("/")[0].rjust(2,"0")


In [61]:
ri.incident_date=ri.incident_date.astype(int)

---
# reCalc: [2010,)

In [171]:
# rige2010=ri[ri.incident_date>=20100101]

In [152]:
bw['#IncidentFrom2010']=-1
rige2010=ri[ri.incident_date>=20100101]
rige2010RID=rige2010.reprisk_id.unique()
for i,j in tqdm(bw.iterrows(),total=bw.shape[0]):
    bwRID=j.repriskid
    DateUB=j.dateAsNum
    if pd.isna(bwRID) or bwRID not in rige2010RID:
        continue
    riFiltered=rige2010[(rige2010.reprisk_id==bwRID) & (rige2010.incident_date<=DateUB)]
    bw.loc[i,'#IncidentFrom2010']=len(riFiltered)
    

100%|██████████| 6500/6500 [00:04<00:00, 1543.68it/s]


In [153]:
bw['#IncidentFrom2010'].value_counts()

-1     5356
 0      432
 1      365
 2      123
 9       68
 3       63
 4       34
 5       22
 10      20
 7        9
 6        4
 8        2
 11       2
Name: #IncidentFrom2010, dtype: int64

In [154]:
6500-5356-432

712

# 3-in-1:

In [195]:
threeInOne=bw[['QuestID','repriskid','AssessmentDate']]
threeInOne.AssessmentDate=threeInOne.AssessmentDate.apply(strYear2Num).astype(int)
threeInOne[['RRIc3Yr','RRIc5Yr','RRIc2010']]=-1

C:\Users\JselZ\AppData\Local\Temp\ipykernel_28604\1937928262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  threeInOne.AssessmentDate=threeInOne.AssessmentDate.apply(strYear2Num).astype(int)
C:\Users\JselZ\AppData\Local\Temp\ipykernel_28604\1937928262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  threeInOne[['RRIc3Yr','RRIc5Yr','RRIc2010']]=-1
C:\Users\JselZ\AppData\Local\Temp\ipykernel_28604\1937928262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [196]:
rige2010RID=rige2010.reprisk_id.unique()
for i,j in tqdm(threeInOne.iterrows(),total=threeInOne.shape[0]):
    bwRID=j.repriskid
    DateUB=int(j.AssessmentDate)
    if pd.isna(bwRID) or bwRID not in rige2010RID:
        continue
    fitRWID=rige2010[(rige2010.reprisk_id==bwRID)]
    riFiltered2010=fitRWID[(fitRWID.incident_date<=DateUB)]
    riFiltered3yr=fitRWID[fitRWID.incident_date.isin(range(DateUB-30000,DateUB+1))]
    riFiltered5yr=fitRWID[fitRWID.incident_date.isin(range(DateUB-50000,DateUB+1))]
    threeInOne.loc[i,'RRIc2010']=len(riFiltered2010)
    threeInOne.loc[i,'RRIc3Yr']=len(riFiltered3yr)
    threeInOne.loc[i,'RRIc5Yr']=len(riFiltered5yr)

100%|██████████| 6500/6500 [00:13<00:00, 484.71it/s]


In [200]:
threeInOne.to_csv("3#Incident.csv",index=0)